# Connect to Indy by IndyDCP

* **robot_ip**: IP address of robot (actually, STEP in IndyCB)
* **name**: string represents robot model
    * Indy7: "NRMK-Indy7"
    * IndyRP2: "NRMK-IndyRP2"
    


In [ ]:
from indy_utils import indydcp_client as client
from indy_utils.indy_program_maker import JsonProgramComponent

import json
import threading
from time import sleep

# Set robot (server) IP 
robot_ip = "192.168.0.111"  # Robot (Indy) IP

# Set robot name
# name = "NRMK-Indy7"  # Robot name (Indy7)
name = "NRMK-Indy3"  # Robot name (IndyRP2)

# Create class object
indy = client.IndyDCPClient(robot_ip, name)

In [ ]:
# Connect to robot
indy.connect()

# Disconnect with robot
# recommanded for finishing program properly
indy.disconnect()

# Examples of JSON Program maker using extended IndyDCP

## Make program example-0: initial and terminal conditions of a program
### Initial statement
* **prog = JsonProgramComponent(policy=0, resume_time=2)**
    * policy: collision policy (0: keep pause, 1: resume after 'resume_time', 2: stop program, 3: no collision detection)
    
### Program body
* **prog.add_move_home()** : go to home position
* **prog.add_move_zero()** : go to zero position

### Terminal statement
* **prog_json = prog.program_done()**  : finish to make program, and return JSON string to be sent to 'indy'
* **indy.set_and_start_json_program(prog_json)** : start the made program
* **indy.set_and_start_json_program(prog.program_done())**  : more simple way

In [ ]:
indy.connect()

indy.set_joint_vel_level(3)

prog = JsonProgramComponent(policy=1, resume_time=2)  # Init. prgoram

prog.add_move_home()
prog.add_move_zero()
prog.add_move_home()

prog_json = prog.program_done()    # Program end

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

In [ ]:
indy.connect()

indy.set_joint_vel_level(3)

prog = JsonProgramComponent(policy=1, resume_time=2)  # Init. prgoram

prog.add_move_home()
prog.add_endtool_do(type=0, value=0)
prog.add_move_zero()
prog.add_wait(1)
prog.add_move_home()
prog.add_endtool_do(type=0, value=1)

prog_json = prog.program_done()

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

## Make program example-0: get teaching positions (joint/task) and save it
* __indy.get_joint_pos()__  : get current joint angle (degree)
* __indy.get_task_pos()__  : get current task-space pose (meter, and degree)

In [ ]:
indy.connect()

In [ ]:
print(indy.get_joint_pos())
print(indy.get_task_pos())

* **indy.update_teaching_data(file_name, waypoint_name, position)**
    * Add teaching points and save it as a file
    * You MUST properly choose joint_move or task_move.
    * We recommand prefix 'j_' or 't_' to represent joint and task

In [ ]:
file_name = 'test.json'
teach_config = indy.update_teaching_data(file_name, 'j_wp1', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp1', indy.get_task_pos())

In [ ]:
teach_config = indy.update_teaching_data(file_name, 'j_wp2', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp2', indy.get_task_pos())

In [ ]:
teach_config = indy.update_teaching_data(file_name, 'j_wp3', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp3', indy.get_task_pos())

In [ ]:
teach_config = indy.update_teaching_data(file_name, 'j_wp4', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp4', indy.get_task_pos())

In [ ]:
indy.disconnect()

## Make program example-1: Joint move with several waypoints
### Arguments: 
* Robot velocity level: 1 - 9
* Joint move blending radius: 3-27 [deg]

In [ ]:
file_name = 'test.json'
teach_config = indy.load_teaching_data(file_name)

teach_config

In [ ]:
j_wp1 = teach_config['j_wp1']
j_wp2 = teach_config['j_wp2']
j_wp3 = teach_config['j_wp3']
j_wp4 = teach_config['j_wp4']

t_wp1 = teach_config['t_wp1']
t_wp2 = teach_config['t_wp2']
t_wp3 = teach_config['t_wp3']
t_wp4 = teach_config['t_wp4']

In [ ]:
# Joint Move
indy.connect()

vel = 5
blend = 20

prog = JsonProgramComponent(policy=0, resume_time=2)                
prog.add_joint_move_to(j_wp1, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp2, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp3, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp4, vel=vel, blend=blend)

prog_json = prog.program_done()
indy.set_and_start_json_program(prog_json)

# indy.disconnect()

In [ ]:
# indy.stop_current_program()
# indy.pause_current_program()
# indy.resume_current_program()

In [ ]:
indy.disconnect()

## Make program example-2: Task move with several waypoints
### Arguments: 
* Robot velocity level: 1 - 9
* Task move blending radius: 0.02 - 0.2 [mm]

In [ ]:
# Task Move
indy.connect()

vel = 5
blend = 0.2

prog = JsonProgramComponent(policy=0, resume_time=2)                
prog.add_task_move_to(t_wp1, vel=vel, blend=blend)
prog.add_task_move_to(t_wp2, vel=vel, blend=blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=blend)

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

## Make program example-3: Use of digital output and tool command


In [ ]:
# Simple example
indy.connect()

vel = 5
j_blend = 20
t_blend = 0.2

prog = JsonProgramComponent(policy=0, resume_time=2)

prog.add_move_home()  
prog.add_joint_move_to(j_wp1, vel=vel) 

# Turns on digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=1)

# Wait for set time
prog.add_wait(1)

# Tool command of tool ID and its command
# Tool should be first set up in Conty Application (Setting - Tool)
# In Conty,add tool and set application (e.g. Pick & Place)
# Edit Hold and Release Output and update the tool (refer the Indy manual)
prog.add_endtool_do(type=0, value=0)
prog.add_wait(1)
prog.add_endtool_do(type=0, value=1)

prog.add_task_move_to(t_wp2, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=t_blend)

# Turns off digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=0)

prog.add_stop()  # Stop program

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

## Make program example-4: Sync motion and async motion
* **indy.wait_for_program_finish() :** this command can synchronize several robot programs

In [ ]:
# Async and sync example
indy.connect()

vel = 5
blend = 20

# Program loop 2 times
for i in range(0,2):
    # Syncronization with Second program after initail loop
    # Wait until Second program is finished with above command (getting postion)
    indy.wait_for_program_finish()

    # First program
    prog = JsonProgramComponent(policy=0, resume_time=2)
    prog.add_joint_move_to(j_wp2, vel=vel, blend=blend)
    prog.add_move_home()
    
    indy.set_and_start_json_program(prog.program_done())    
    
    # Asyncronization with Second program
    # Wait until First program is finished, after then below program is executed
    indy.wait_for_program_finish()

    # Second program
    prog = JsonProgramComponent(policy=0, resume_time=2)                
    prog.add_task_move_to(t_wp3, vel=vel, blend=blend)
    prog.add_move_home()

    indy.set_and_start_json_program(prog.program_done())
    
indy.disconnect()

## Make program example-5: Multi-threading for interfacing other devices (buttons, sensors,...)

In [ ]:
indy.connect()

In [ ]:
# Global variables
GLOBAL_INDICATOR = {'position': True, 'dio': True, 'program': True, 'terminated': False}


# Monitoring current robot position at every 1 sec 
def monitor_robot_position():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['position']:
        if GLOBAL_INDICATOR['terminated']:
            break
        sleep(0.1)
        j_pos = indy.get_joint_pos()
        t_pos = indy.get_task_pos()
        print("Joint angles: ", j_pos)
        print("Task pose", t_pos)        
        
# Monitoring digital input at every 1 sec 
def monitor_dio():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['dio']:
        if GLOBAL_INDICATOR['terminated']:
            break
        sleep(0.1)
        dis = indy.get_di()
        
        btn1 = dis[1]
        btn2 = dis[3]
        btn3 = dis[5]
        print("Button 1/2/3 = [{}, {}, {}]".format(btn1, btn2, btn3))
        if btn1 == 1:
            # TODO: implement an action when button1 on
            pass
        if btn2 == 1:
            # TODO: implement an action when button2 on
            pass
        if btn3 == 1:
            # TODO: implement an action when button3 on
            pass

# Inifinity loop of robot program 
def run_program():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['program']:        
        if GLOBAL_INDICATOR['terminated']:
            break
        
        prog = JsonProgramComponent(policy=0, resume_time=2)
        
        prog.add_move_home()
        prog.add_move_zero()
        
        indy.set_and_start_json_program(prog.program_done())
        indy.wait_for_program_finish()
        
    GLOBAL_INDICATOR['position'] = False
    GLOBAL_INDICATOR['dio'] = False
    GLOBAL_INDICATOR['program'] = False
    
        
# Create each thread for 'run_program' and 'get_current_position'
th1 = threading.Thread(target=run_program)  # thread for 'run_program'
th2 = threading.Thread(target=monitor_robot_position)  # thread for 'get_current_position'
th3 = threading.Thread(target=monitor_dio)  # thread for 'get_current_position'

th3.start()
th2.start()
th1.start()

In [ ]:
# Stop above program
indy.stop_motion()

GLOBAL_INDICATOR['position'] = False
GLOBAL_INDICATOR['dio'] = False
GLOBAL_INDICATOR['program'] = False
GLOBAL_INDICATOR['terminated'] = True

In [ ]:
indy.disconnect()

## Other functions: Direct variables

In [ ]:
# Simple variable assignment example
indy.connect()

# Variables
var_name=["ab","bc","ca"]
var_value=[1,2,3]
var_type=[1,1,1]

prog = JsonProgramComponent(policy=0, resume_time=2, var_name=var_name, var_value=var_value, var_type=var_type)

prog.add_move_zero()
prog.add_var_assign(name="ab", val=100, type=1)  # Variable declaration
prog.add_analog_out(idx=0, val=0)

prog.add_wait_for(time=3, left_type=10, left_value="ab", right_type=1, right_value=100, op=0)  # Wait for variable assigned to other value
prog.add_move_home()

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

# Practice now ! 